# Vehicle Detection and Tracking

## Step1: Import all packages

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from lesson_functions import *
# NOTE: the next import is only valid for scikit-learn version <= 0.17
# for scikit-learn >= 0.18 use:
# from sklearn.model_selection import train_test_split
from sklearn.cross_validation import train_test_split

from scipy.ndimage.measurements import label
import pickle


In [ ]:
### parameters
color_space = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"
spatial_size = (16, 16) # Spatial binning dimensions
hist_bins = 16    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off
y_start_stop = [400, 720]  # Min and max in y to search in slide_window()

In [ ]:
# read training data
#glob.iglob('src/**/*.c', recursive=True)
cars = glob.glob('./training_images/vehicles/**/*.png')
notcars = glob.glob('./training_images/non-vehicles/**/*.png')
print(len(cars))
print(len(notcars))

In [ ]:
# extract training features
t=time.time()
car_features = extract_features(cars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
notcar_features = extract_features(notcars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)

t2 = time.time()
print(round(t2-t, 2), 'Seconds to extract features...')

# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=rand_state)

# Fit a per-column scaler
X_scaler = StandardScaler().fit(X_train)
# Apply the scaler to X
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))
print('train set size:', len(X_train))
print('test set size:', len(X_test))

In [ ]:
# training
# Use a linear SVC 
svc = LinearSVC(C=100, dual=False)
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Training Accuracy of SVC = ', round(svc.score(X_train, y_train), 4))
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()

In [ ]:
from sklearn import linear_model
clf = linear_model.SGDClassifier(alpha= 1e-6,penalty='l2',loss='hinge',max_iter=1000,n_jobs=-1,tol=1e-7)
t=time.time()
clf.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SGD...')
# Check the score of the SVC
print('Training Accuracy of SGD = ', round(clf.score(X_train, y_train), 4))
print('Test Accuracy of SGD = ', round(clf.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()

In [ ]:
import xgboost as xgb
from xgboost import plot_importance


params = {
    'booster': 'gbtree',
    'objective': 'multi:softmax',
    'num_class': 2,
    'gamma': 0.1,
    'max_depth': 6,
    'lambda': 2,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'min_child_weight': 3,
    'silent': 1,
    'eta': 0.1,
    'seed': 1000,
    'nthread': 4,
}

plst = params.items()

In [ ]:
dtrain = xgb.DMatrix(X_train, y_train)
num_rounds = 40
model = xgb.train(plst, dtrain, num_rounds)

In [ ]:
model.save_model("xgbmodel")

In [ ]:
num_rounds = 10
model = xgb.train(plst, dtrain, num_rounds, xgb_model='xgbmodel')

In [ ]:
model = xgb.Booster(plst)
model.load_model("xgbmodel")

In [ ]:

# predict
dtest = xgb.DMatrix(X_train)
ans = model.predict(dtest)

# accuracy
cnt1 = 0
cnt2 = 0
for i in range(len(y_train)):
    if ans[i] == y_train[i]:
        cnt1 += 1
    else:
        cnt2 += 1

print(dtest.num_col())
print(dtest.num_row())

print("Training Accuracy: %.2f %% " % (100 * cnt1 / (cnt1 + cnt2)))

In [ ]:
# 对测试集进行预测
dtest = xgb.DMatrix(X_test)
ans = model.predict(dtest)

# 计算准确率
cnt1 = 0
cnt2 = 0
for i in range(len(y_test)):
    if ans[i] == y_test[i]:
        cnt1 += 1
    else:
        cnt2 += 1

print(dtest.num_col())
print(dtest.num_row())
print("Test Accuracy: %.2f %% " % (100 * cnt1 / (cnt1 + cnt2)))

In [ ]:
# 显示重要特征
plot_importance(model)
plt.show()

In [ ]:
# training
# Use a linear SVC 

linear_svc=svm.SVC(C=100,gamma='auto',kernel='linear')

# Check the training time for the SVC
t=time.time()
linear_svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train linear SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(linear_svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()

In [ ]:
# training
# Use a rbf SVC 

rbf_svc=svm.SVC(kernel='rbf',C=1e-3)

# Check the training time for the SVC
t=time.time()
rbf_svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train rbf SVC...')
# Check the score of the SVC
print('Training Accuracy of SVC = ', round(rbf_svc.score(X_train, y_train), 4))
print('Test Accuracy of SVC = ', round(rbf_svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()

In [ ]:
# save the params
data = {
    'svc': svc,
    'X_scaler': X_scaler
}
with open('data.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
# restore params
dist_pickle = pickle.load( open("16x16_data.pickle", "rb" ) )
svc = dist_pickle["svc"]
X_scaler = dist_pickle["X_scaler"]

In [ ]:
# detect
y_start_stop = [400, 500]  # Min and max in y to search in slide_window()
image = mpimg.imread('test_images/test6.jpg')
draw_image = np.copy(image)

image = image.astype('float32')/255

# Uncomment the following line if you extracted training
# data from .png images (scaled 0 to 1 by mpimg) and the
# image you are searching is a .jpg (scaled 0 to 255)
#image = image.astype(np.float32)/255

windows = slide_window(image, x_start_stop=[None, None], y_start_stop=y_start_stop, 
                    xy_window=(64, 64), xy_overlap=(0.75, 0.75))

#print(len(windows))
hot_windows = search_windows(image, windows, model, X_scaler, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)                       
#print(len(hot_windows))
window_img = draw_boxes(draw_image, hot_windows, color=(0, 0, 255), thick=6)                    

plt.imshow(window_img)
plt.show()

In [ ]:
draw_image = np.copy(image)

heat = np.zeros_like(image[:,:,0]).astype(np.float)
add_heat(heat, hot_windows)
# Apply threshold to help remove false positives
heat = apply_threshold(heat, 1)

# Visualize the heatmap when displaying    
heatmap = np.clip(heat, 0, 255)

# Find final boxes from heatmap using label function
labels = label(heatmap)
    
draw_labeled_bboxes(draw_image, labels)
#plt.imshow(draw_image)
#plt.show()

figure, (detected_car_image_plot, detected_car_heatmap_plot, detected_car_labales_plot) = plt.subplots(1, 3, figsize=(20,15))
        
detected_car_image_plot.set_title('Detected cars')
detected_car_image_plot.imshow(draw_image)

detected_car_heatmap_plot.set_title('Heatmap')
detected_car_heatmap_plot.imshow(heatmap)

detected_car_labales_plot.set_title('Labels')
detected_car_labales_plot.imshow(labels[0], cmap='gray')
plt.show()


In [ ]:
# Define a single function that can extract features using hog sub-sampling and make predictions
def find_cars(img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins):
    
    #draw_img = np.copy(img)
    img = img.astype(np.float32)/255.0
    
    img_tosearch = img[ystart:ystop,:,:]
    ctrans_tosearch = convert_color(img_tosearch, conv='RGB2YCrCb')
    #ctrans_tosearch = img_tosearch
    #ctrans_tosearch = img
    if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
    
    ch1 = ctrans_tosearch[:,:,0]
    ch2 = ctrans_tosearch[:,:,1]
    ch3 = ctrans_tosearch[:,:,2]

    # Define blocks and steps as above
    nxblocks = (ch1.shape[1] // pix_per_cell) - cell_per_block + 1
    nyblocks = (ch1.shape[0] // pix_per_cell) - cell_per_block + 1 
    nfeat_per_block = orient*cell_per_block**2
    
    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    window = 64
    nblocks_per_window = (window // pix_per_cell) - cell_per_block + 1
    cells_per_step = 2  # Instead of overlap, define how many cells to step
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step + 1
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step + 1
    #print(nxsteps)
    #print(nysteps)
    # Compute individual channel HOG features for the entire image
    hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)
    
    hot_windows = []
    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step
            # Extract HOG for this patch
            hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))

            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell

            # Extract the image patch
            subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (64,64))
          
            # Get color features
            spatial_features = bin_spatial(subimg, size=spatial_size)
            hist_features = color_hist(subimg, nbins=hist_bins)

            # Scale features and make a prediction
            test_features = X_scaler.transform(np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1))  
            #test_prediction = svc.predict(test_features)
            dtest = xgb.DMatrix(test_features)
            test_prediction = model.predict(dtest)
            
            if test_prediction == 1:
            #if test_decision > 0.7:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                hot_windows.append(((xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart)))
                #cv2.rectangle(draw_img,(xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart),(0,0,255),6) 
    
    #draw_labeled_bboxes(draw_img, labels)
    
    return hot_windows

In [ ]:
history_heat = []
MAX_HISTORY = 10
#history_heat = []

In [ ]:
def pipeline(img):
    hot_windows1 = []
    hot_windows2 = []
    hot_windows3 = []
    hot_windows4 = []

    hot_windows1 = find_cars(img, 400, 500, 1, model, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)
    #hot_windows2 = find_cars(img, 380, 530, 1.1, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)
    hot_windows2 = find_cars(img, 450, 530, 1.5, model, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)
    hot_windows3 = find_cars(img, 380, 530, 2, model, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)

    #hot_windows1 = find_cars(img, 380, 530, 1, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)
    #plt.imshow(out_img)
    #plt.show()
    #hot_windows2 = find_cars(img, 380, 550, 1.5, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)
    #plt.imshow(out_img)
    #plt.show()
    #hot_windows3 = find_cars(img, 380, 530, 2, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)
    #plt.imshow(out_img)
    #plt.show()
    #out_img,hot_windows1 = find_cars(img, 380, 530, 1, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)
    #plt.imshow(out_img)
    #plt.show()
    #out_img,hot_windows1 = find_cars(img, 380, 530, 1, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)
    #plt.imshow(out_img)
    #plt.show()
    draw_img = np.copy(img)

    hwin = hot_windows1 + hot_windows2 + hot_windows3
    #hwin = hot_windows1


    heat = np.zeros_like(img[:,:,0]).astype(np.float)
    add_heat(heat, hwin)
    #heat = apply_threshold(heat, 0)
    heat = apply_threshold(heat, 2)


    
    if len(history_heat) < MAX_HISTORY:
        history_heat.append(heat)
    else:
        del history_heat[0]
        history_heat.append(heat)
        

    total_heat = np.zeros_like(img[:,:,0]).astype(np.float)
    for h in history_heat:
        total_heat += h

    # Apply threshold to help remove false positives
    heat_sum = apply_threshold(total_heat, 32)
    #heat_sum = apply_threshold(total_heat, 2)


    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat_sum, 0, 255)
    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    draw_labeled_bboxes(draw_img, labels)
    
    #plt.imshow(heat_sum, cmap='hot')
    #plt.show()


    return draw_img

In [ ]:
img = mpimg.imread('./test_images/test3.jpg')
out_img = pipeline(img)
plt.imshow(out_img)
plt.show()

In [ ]:
from moviepy.editor import VideoFileClip
def detectVideo(input_video,output):
    clip1 = VideoFileClip(input_video)
    out_clip = clip1.fl_image(pipeline)
    out_clip.write_videofile(output,audio=False)

#detectVideo('test_video.mp4','test_video_output.mp4')
#project_video.mp4
detectVideo('project_video.mp4','project_video_out.mp4')


In [ ]:
from moviepy.editor import VideoFileClip
history_heat = []
def clipVideo(input_video,output,start,end):
    clip1 = VideoFileClip(input_video)
    clip1 = clip1.subclip(start,end)
    out_clip = clip1.fl_image(pipeline)
    out_clip.write_videofile(output,audio=False)



#clipVideo('test_video.mp4','test_video_clip.mp4',0,1)
clipVideo('project_video.mp4','project_video_clip.mp4',44,47)